In [ ]:
pip install demoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 3.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import re
import demoji
import string
import nltk
nltk.download('punkt')
from textblob import TextBlob

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
df = pd.read_csv('/content/omicron.csv')
df.sample(5)

,id,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,retweets,favorites,is_retweet
12387,1496572008213413888,Sai Naveena K,"Tirupati, India",I hope that my impression about myself and you...,2012-12-21 17:21:08,63,2,160,False,2022-02-23 19:45:46,"Omicron variant silent killer, recovery takes ...","['Omicron', 'longrecovery']",Twitter for Android,0,0,False
3868,1493593159414292487,Tim White,The North of England,Data & Covid expert. Journalist by trade. Nowa...,2011-03-18 17:23:04,15704,570,28214,False,2022-02-15 14:28:53,#Covid19 cases in #Latvia also down today for ...,"['Covid19', 'Latvia', 'Omicron']",Twitter Web App,1,3,False
14454,1499118183772528641,"Yan Leyfman, MD",NaN,#Medicine Researcher in #Oncology #COVID19 #Im...,2020-04-08 21:08:25,1399,1569,26742,False,2022-03-02 20:23:21,"Conclusion: Thus, #COVID19 #booster #immunizat...","['COVID19', 'booster', 'immunizations', 'omicr...",Twitter Web App,1,1,False
6773,1493981301484888069,Marty Freetage,"Scottsdale, AZ","During this pandemic of COVID-19, I want to pr...",2020-11-11 12:52:58,222,358,1324,False,2022-02-16 16:11:13,Happy Hump Day! Here are today’s numbers for C...,"['coronavirus', 'vaccine', 'COVID19', 'breaking']",Twitter Web App,0,3,False
12491,1496533464476176397,TalkRadio 77 WABC,"New York, NY",The official account of TALKRADIO 77 WABC \nNe...,2010-01-13 18:40:31,22064,673,3380,True,2022-02-23 17:12:36,US Vaccination Drive is Bottoming Out as Omicr...,"['77WABCRadio', 'vaccination', 'Omicron']",Twitter Web App,0,0,False


In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('turkish'))
print(stop_words)

{'mı', 'nereye', 'acaba', 'ile', 'bu', 'o', 'siz', 'biz', 'şey', 'nerde', 'hepsi', 'da', 'diye', 'biri', 'en', 'ya', 'nerede', 'belki', 'ne', 'niçin', 'kez', 'ise', 'eğer', 'de', 'aslında', 'gibi', 'için', 'tüm', 'niye', 'daha', 'veya', 'nasıl', 'ama', 'bazı', 'kim', 'sanki', 'birşey', 'hep', 'birkaç', 'her', 'mu', 'hem', 'yani', 'şu', 've', 'çok', 'hiç', 'defa', 'çünkü', 'neden', 'az', 'mü', 'ki'}


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def cleanTweet(tweet):
    tweet = tweet.lower()
    tweet = demoji.replace(tweet,"")
    tweet = re.sub('\n',' ',tweet)
    tweet = re.sub(r'@\w+|\#','',tweet)
    tweet = tweet.translate(str.maketrans("","",string.punctuation))
    tweet = re.sub(r"http\s+|www\S+|https\S+","",tweet,flags=re.MULTILINE) # remove link
    filtered_words = [word for word in TextBlob(tweet).words if word not in stop_words]
    return " ".join(filtered_words)

In [ ]:
df['cleanTweet'] = df['text'].map(cleanTweet)
df.sample(5)

,id,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,retweets,favorites,is_retweet,cleanTweet
14073,1496006663434047491,Rakyat Merdeka,NaN,official akun Rakyat Merdeka Digital,2021-12-23 13:39:54,16,3,0,False,2022-02-22 06:19:17,Menhan Malaysia Hishammuddin Hussein Positif C...,"['Malaysia', 'Omicron', 'OmicronVariant', 'Cor...",Twitter for Android,4,4,False,menhan malaysia hishammuddin hussein positif c...
6679,1493995786165571586,Nathan Joyner,"Los Angeles, CA",Global Venture Captial and Private Equity/Busi...,2015-05-18 20:52:29,57,11,816,False,2022-02-16 17:08:47,Daily Confirmed Covid Cases per 1K Population ...,NaN,smcapplication,1,1,False,daily confirmed covid cases per 1k population ...
13707,1496119910703017986,Smbat Gevorgyan,NaN,PhD in Medical Nanotechnology,2019-05-07 06:41:47,65,89,499,False,2022-02-22 13:49:17,The #Omicron wave (the last peak in the graph)...,"['Omicron', 'Armenia']",Twitter Web App,0,3,False,the omicron wave the last peak in the graph is...
15674,1500835862766104579,Patrizia Roberto,"Glasgow, Scotland","Mum, daughter, sister, friend, Scottish indepe...",2012-03-01 21:57:45,1204,1742,21915,False,2022-03-07 14:08:48,Is everyone registering their positive lateral...,NaN,Twitter for iPhone,1,0,False,is everyone registering their positive lateral...
7640,1494915280010547202,𝙽𝚊𝚝𝚎 𝙲𝚘𝚌𝚑𝚛𝚊𝚗𝚎,"Luna, Aitken Basin","""Ex astris ad mortem."" I acknowledge tradition...",2008-11-28 22:39:28,10528,9932,7589,True,2022-02-19 06:02:31,👀 \n\nDon’t let anyone minimise how you feel. ...,"['OmicronIsNotMild', 'Omicron']",Twitter for iPhone,3,6,False,don ’ t let anyone minimise how you feel ever ...


In [ ]:
import gensim
from gensim.utils import simple_preprocess

In [ ]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

In [ ]:
data = df['cleanTweet'].to_list()
data_words = list(sent_to_words(data))

In [ ]:
print(data_words[:1][0][:30])

['daily', 'us', 'confirmed', 'covid', 'cases', 'by', 'county', 'for', 'me', 'covid', 'insights', 'from', 'our', 'analytics', 'team', 'and', 'usafacts']


In [ ]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [ ]:
import gensim.corpora as corpora

# Create Dictionary
id2word = corpora.Dictionary(data_words)

# Create Corpus
texts = data_words

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1][0][:30])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 2), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1)]


In [ ]:
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=10,
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)

In [ ]:
from pprint import pprint
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.209*"cases" + 0.129*"covid" + 0.111*"new" + 0.036*"in" + 0.030*"india" + '
  '0.028*"today" + 0.027*"total" + 0.026*"death" + 0.022*"daily" + '
  '0.020*"hours"'),
 (1,
  '0.059*"how" + 0.044*"vaccine" + 0.042*"against" + 0.037*"he" + 0.035*"to" + '
  '0.034*"news" + 0.028*"would" + 0.028*"infection" + 0.015*"antibody" + '
  '0.014*"me"'),
 (2,
  '0.060*"going" + 0.027*"rate" + 0.026*"here" + 0.023*"less" + 0.021*"does" + '
  '0.019*"hospitalization" + 0.018*"while" + 0.014*"remember" + 0.014*"doesnt" '
  '+ 0.014*"watch"'),
 (3,
  '0.083*"omicron" + 0.051*"of" + 0.048*"the" + 0.044*"ba" + 0.041*"covid" + '
  '0.038*"in" + 0.021*"on" + 0.021*"more" + 0.020*"variant" + 0.017*"and"'),
 (4,
  '0.035*"or" + 0.027*"omicron" + 0.026*"covid" + 0.022*"with" + 0.022*"your" '
  '+ 0.021*"be" + 0.020*"study" + 0.020*"two" + 0.016*"can" + 0.016*"may"'),
 (5,
  '0.064*"you" + 0.062*"to" + 0.054*"it" + 0.038*"that" + 0.032*"will" + '
  '0.026*"don" + 0.024*"up" + 0.024*"omicron" + 0.022*"j

In [ ]:
from gensim.models import CoherenceModel

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_words, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

Coherence Score:  0.40453933603176423


In [ ]:
# supporting function
def compute_coherence_values(corpus, dictionary, k, a, b):

    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k,
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)

    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_words, dictionary=id2word, coherence='c_v')

    return coherence_model_lda.get_coherence()

In [ ]:
import numpy as np
import tqdm

grid = {}
grid['Validation_Set'] = {}

# Topics range
min_topics = 2
max_topics = 11
step_size = 1
topics_range = range(min_topics, max_topics, step_size)

# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')

# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')

# Validation sets
num_of_docs = len(corpus)
corpus_sets = [gensim.utils.ClippedCorpus(corpus, int(num_of_docs*0.75)),
               corpus]

corpus_title = ['75% Corpus', '100% Corpus']

model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }

In [ ]:
num_topics = 8

lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=num_topics,
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=0.01,
                                           eta=0.9)

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation as LDA
from sklearn.feature_extraction.text import CountVectorizer

tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=10000)
tf = tf_vectorizer.fit_transform(data)
tf_feature_names = tf_vectorizer.get_feature_names_out()

In [ ]:
lda = LDA(n_components=30, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(tf)

In [ ]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic %d:" % (topic_idx))
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))
display_topics(lda,tf_feature_names,20)

Topic 0:
vaccine antibody 2021 to original living december putin backgroundimmunity friend for vs coverage critical 95 omicronspecific teachers becomes cbc protest
Topic 1:
against out infection omicron and vaccines protection sarscov2 through vaccination previous find head boosters check closed worldwide efficacy reduce doesnt
Topic 2:
miyapur udbhava swabs radioactive technetium cevicas shri ba2h78y board memory kphb 212 omicronba2 abledsupremacys bell oxford par udbhavahospitals markedly noncovid
Topic 3:
study by omicron infections booster news good to covid19 say between not are effectiveness spreading enough am use info estimates
Topic 4:
live time looks dying working restrictions up qingdao email went freedom ended with canadahasfallen mail premier summer fifth employees settings
Topic 5:
food department games continuing cuba npr st explain health covid19nz mobile families cm kirby paying praying floral clustering gazpacho cut
Topic 6:
emergency paypal public hollandfrance33com 